## Leave-One-Out Cross-Validation

In this notebook we are going to use LOOCV to assess the out-of-sample accuracy of two regression models (linear and quadratic regression).
We will use sklearn's `cross_val_score`, which takes care of all the tasks associated with LOOCV:
* splitting the dataset,
* iterating over all the points to choose a different test point at each iteration,
* training the model on the remaining $n-1$ points,
* and finally compute the model's error on the left-out point.
This function returns an array with all the $n$ test MSEs it computed.
To get the final MSE of our model, we average these $n$ values.

**Technical note**: As a design decision by sklearn's authors, `cross_val_score` wants a *scoring* function, i.e., a function which is higher when the model performs better and lower when the model performs poorly. The MSE, on the other hand, is a *loss* function: it is low for good models and high for bad models. The simplest way to turn a loss function into a scoring function is to take its negative (multiply by -1). That's why the scoring function is referref to as `neg_mean_squared_error` (*neg* stands for negative).

In [1]:
import pandas as pd

In [2]:
d = pd.read_csv('auto-mpg.csv')

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import LeaveOneOut, cross_val_score
import numpy as np

In [4]:
X = d.drop('mpg', axis=1)
y = d.mpg

### Creating the two models using sklearn's pipelines

In [5]:
linear_model = make_pipeline(
    StandardScaler(),
    LinearRegression()
)

In [6]:
quadratic_model = make_pipeline(
    PolynomialFeatures(degree=2),
    StandardScaler(),
    LinearRegression()
)

### Perform LOOCV and average the (negative) scores

Note how we use `cv=LeaveOneOut()` to tell `cross_val_score` that we want to use LOOCV as our cross-validation method.

In [7]:
sol_linear = cross_val_score(
    estimator=linear_model,
    X=X, y=y,
    scoring='neg_mean_squared_error',
    cv=LeaveOneOut())

In [8]:
-1 * np.mean(sol_linear)

11.371126332686618

In [9]:
sol_quadratic = cross_val_score(
    estimator=quadratic_model,
    X=X, y=y,
    scoring='neg_mean_squared_error',
    cv=LeaveOneOut())

In [10]:
-1 * np.mean(sol_quadratic)

13.48894067034972

It looks like the linear regression model has a lower error: it is the model we will use in production!

### Retraining the winner model on the entire dataset

In [11]:
winner = linear_model.fit(X, y)